In [1]:
import xarray as xr
import numpy as np
from geocube.vector import vectorize
da = xr.open_dataset(r"D:\Datasets\Köppen-Geiger Climate Classification\KG_1986-2010.grd", engine="rasterio").band_data.sel(band=1)

# To geopandas
gdf = vectorize(da)

# Drop ocean
gdf = gdf[gdf.band_data != 32]

# Assign categories
categories_32 = ['Af', 'Am', 'As', 'Aw', 'BSh', 'BSk', 'BWh', 'BWk', 'Cfa', 'Cfb','Cfc', 'Csa', 'Csb', 'Csc', 'Cwa','Cwb', 'Cwc', 'Dfa', 'Dfb', 'Dfc','Dfd', 'Dsa', 'Dsb', 'Dsc', 'Dsd','Dwa', 'Dwb', 'Dwc', 'Dwd', 'EF','ET', 'Ocean']
labels_32 = dict(zip(range(1, 32+1), categories_32))

labels_5 = {"A":"Equatorial", "B":"Arid", "C":"Warm Temperate", "D":"Boreal", "E":"Polar"}

gdf['climate_band_32'] = gdf['band_data'].map(labels_32)
gdf['climate_band_5_data'] = gdf['climate_band_32'].str[0]
gdf['climate_band_5'] = gdf['climate_band_5_data'].map(labels_5)

# Show
# gdf.plot(column="climate_band_5")

In [2]:
import pandas as pd
import geopandas as gpd

df = pd.read_stata(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_in\DHS\DHSBirthsGlobalAnalysis_05142024.dta")
gdf_dhs = df[["ID_HH","LATNUM","LONGNUM"]].drop_duplicates(subset="ID_HH")
gdf_dhs = gpd.GeoDataFrame(gdf_dhs, geometry=gpd.points_from_xy(gdf_dhs["LONGNUM"], gdf_dhs["LATNUM"]))
# gdf_dhs.plot()

In [ ]:
gdf

In [3]:
gdf_dhs = gdf_dhs.set_crs("epsg:4326", allow_override=True)
gdf = gdf.set_crs("epsg:4326", allow_override=True)
gdf_dhs_climatebands = gdf_dhs.sjoin(gdf)

In [20]:
gdf_dhs_climatebands[["ID_HH", "climate_band_32", "climate_band_5_data", "climate_band_5"]].drop_duplicates("ID_HH").to_stata(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\DHSBirthsGlobalAnalysis_05142024_climate_bands_assigned.dta")

In [ ]:
gdf_dhs_climatebands[gdf_dhs_climatebands.ID_HH.duplicated(keep=False)]